In [ ]:
from __future__ import print_function
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, Input
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import backend as K
from keras.layers.advanced_activations import LeakyReLU
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import seaborn as sys
import pandas as pd
import tensorflow as tf
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import roc_auc_score

In [ ]:
batch_size = 128
num_class = 10
epoch = 25

r,c = 28,28
(x_train,y_train),(x_test,y_test) = mnist.load_data()
x_test_test = x_test.copy()

In [ ]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0],1, r,c)
    x_test = x_test.reshape(x_test.reshape[0],1,r,c)
    ip = (1,r,c)
else:
    x_train = x_train.reshape(x_train.shape[0], r,c,1)
    x_test = x_test.reshape(x_test.shape[0], r,c,1)
    ip = (r,c,1)

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print('x_train.shape[0]',x_train.shape[0])
print('x_test.shape[0]',x_test.shape[0])
print('x_train',x_train.shape)
print('y_train',y_train.shape)

In [ ]:
y_train = keras.utils.to_categorical(y_train, num_class)
y_test = keras.utils.to_categorical(y_test, num_class)

In [ ]:
print(y_train.shape)
print(y_test.shape)

In [ ]:
m = Sequential()
m.add(Conv2D(32,kernel_size=(3,3),input_shape=ip))
m.add(LeakyReLU(alpha=0.001))
m.add(MaxPooling2D(pool_size=(2,2)))
m.add(BatchNormalization())

m.add(Conv2D(64,kernel_size=(3,3)))
m.add(LeakyReLU(alpha=0.001))
m.add(MaxPooling2D(pool_size=(2,2)))
m.add(BatchNormalization())

m.add(Conv2D(128,kernel_size=(3,3)))
m.add(LeakyReLU(alpha=0.001))
m.add(MaxPooling2D(pool_size=(2,2)))
m.add(BatchNormalization())

m.add(Flatten())
m.add(Dense(1024))
m.add(BatchNormalization())
m.add(LeakyReLU(alpha=0.001))
m.add(Dropout(0.50))

m.add(Dense(128))
m.add(BatchNormalization())
m.add(LeakyReLU(alpha=0.001))
m.add(Dropout(0.25))

m.add(Dense(num_class,activation='softmax'))

In [ ]:
m.summary()

In [ ]:
m.compile(loss=keras.losses.categorical_crossentropy,
         optimizer=keras.optimizers.Adam(),
         metrics=['accuracy'])

his = m.fit(x_train,y_train,
     batch_size=batch_size,
     epochs=10,
     verbose=1,
     validation_data=(x_test,y_test))

In [ ]:
plt.plot(his.history["accuracy"])
plt.plot(his.history["val_accuracy"])
plt.title("Model_Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("epochs")
plt.legend(['Train',"Test"],loc="upper left")
plt.show()

In [ ]:
plt.plot(his.history["loss"])
plt.plot(his.history["val_loss"])
plt.title("Model_loss")
plt.ylabel("Loss")
plt.xlabel("epochs")
plt.legend(['Train',"Test"],loc="upper left")
plt.show()

In [ ]:
score = m.evaluate(x_test,y_test,verbose=0)
print('Test loss:', score[0])
print('Test accuracy:',score[1])

In [ ]:
y_pred = m.predict(x_test)
y_pred_one_hot = np.argmax(y_pred,axis=1)
y_true = np.argmax(y_test,axis=1)

In [ ]:
cofm = confusion_matrix(y_true=y_true, y_pred=y_pred_one_hot)
print(cofm)

In [ ]:
clr = classification_report(y_true=y_true,y_pred=y_pred_one_hot)
print(clr)

In [ ]:
con_mat = tf.math.confusion_matrix(labels=y_true, predictions=y_pred_one_hot).numpy()
print(con_mat)

In [ ]:
cf_mx_norm = np.around(con_mat.astype("float")/con_mat.sum(axis=1)[:,np.newaxis],decimals=2)

In [ ]:
print(cf_mx_norm)

In [ ]:
class_num = ['0','1','2','3','4','5','6','7','8','9']
cf_mx_norm_df = pd.DataFrame(cf_mx_norm, index=class_num, columns=class_num)

In [ ]:
cf_mx_norm_df

In [ ]:
figure = plt.figure(figsize=(8,8))
sys.heatmap(cf_mx_norm_df,annot=True, cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel("True_label")
plt.xlabel("Predicted_label")
plt.show()

In [ ]:
from sklearn.preprocessing import LabelBinarizer
def multiclass_roc_auc_score(y_test, y_pred, all_labels,average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)

    for (idx, c_label) in enumerate(all_labels): # all_labels: no of the labels
        fpr, tpr, thresholds = roc_curve(y_test[:,idx].astype(int), y_pred[:,idx])
        auc_score = auc(fpr, tpr)
        plt.figure()
        plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score)
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example')
        plt.legend(loc="lower right")
        plt.show()

    return roc_auc_score(y_test, y_pred, average=average)
y_pred = m.predict(x_test,verbose=True)
y_pred = np.argmax(y_pred,axis=1)
y_test = np.argmax(y_test,axis=1)
class_num = ['0','1','2','3','4','5','6','7','8','9']
multiclass_roc_auc_score(y_test,y_pred,class_num)    

In [ ]:
classf = m.predict_classes(x_test,batch_size=10)


In [ ]:
print("classf",classf[10])

In [ ]:
layer_outputs = [layer.output for layer in m.layers[:12]] 
activation_model = Model(inputs=m.input, outputs=layer_outputs) 

layer_outputs = [layer.output for layer in m.layers[:12]] 
activation_model = Model(inputs=m.input, outputs=layer_outputs) 

In [ ]:
img = np.expand_dims(x_test[10],axis=0)
img.shape

In [ ]:
act = activation_model.predict(img)

In [ ]:
fir_lay_act = act[0]

In [ ]:
plt.matshow(fir_lay_act[0,:,:,4],cmap='viridis')
plt.show()

In [ ]:
layer_names = []
for layer in m.layers[:12]:
    layer_names.append(layer.name) 
    
images_per_row = 16


In [ ]:
for layer_name, layer_activation in zip(layer_names, act): # Displays the feature maps
    n_features = layer_activation.shape[-1] # Number of features in the feature map
    size = layer_activation.shape[1] #The feature map has shape (1, size, size, n_features).
    n_cols = n_features // images_per_row # Tiles the activation channels in this matrix
    display_grid = np.zeros((size * n_cols, images_per_row * size))
    for col in range(n_cols): # Tiles each filter into a big horizontal grid
        for row in range(images_per_row):
            channel_image = layer_activation[0,
                                             :, :,
                                             col * images_per_row + row]
            channel_image -= channel_image.mean() # Post-processes the feature to make it visually palatable
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size, # Displays the grid
                         row * size : (row + 1) * size] = channel_image
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')
    

In [ ]:
def vis(img):
    layer_outputs = [layer.output for layer in m.layers[:12]] 
    activation_model = Model(inputs=m.input, outputs=layer_outputs) 

    layer_outputs = [layer.output for layer in m.layers[:12]] 
    activation_model = Model(inputs=m.input, outputs=layer_outputs) 
    act = activation_model.predict(img)
    layer_names = []
    for layer in m.layers[:12]:
        layer_names.append(layer.name) 

    images_per_row = 16
    
    for layer_name, layer_activation in zip(layer_names, act): 
        n_features = layer_activation.shape[-1] 
        size = layer_activation.shape[1] 
        n_cols = n_features // images_per_row 
        display_grid = np.zeros((size * n_cols, images_per_row * size))
        for col in range(n_cols): 
            for row in range(images_per_row):
                channel_image = layer_activation[0,
                                                 :, :,
                                                 col * images_per_row + row]
                channel_image -= channel_image.mean() 
                channel_image /= channel_image.std()
                channel_image *= 64
                channel_image += 128
                channel_image = np.clip(channel_image, 0, 255).astype('uint8')
                display_grid[col * size : (col + 1) * size, # Displays the grid
                             row * size : (row + 1) * size] = channel_image
        scale = 1. / size
        plt.figure(figsize=(scale * display_grid.shape[1],
                            scale * display_grid.shape[0]))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='viridis')
        

In [ ]:
img = np.expand_dims(x_test[10],axis=0)
vis(img)